# ASAM ODS Playground

This notebook provides a simple query and mass data example you can change and extend to your needs.

Use VSCode Data Wrangler or Google Colab to experiment with the data. [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg "Open in Colab")](https://colab.research.google.com/github/peak-solution/data_management_learning_path/blob/main/data_management_learning_path/ods/playground.ipynb)

# Initialize and connect

The [ASAM ODSBox](https://pypi.org/project/odsbox/) contain some functionality that wraps the ODS HTTP API making using Python easier  ;-)

It contains:
- http wrapper implemented using protobuf and requests
- utility that converts ODS DataMatrices into pandas.DataFrame

The ODS HTTP API is a session based API. The session ID is called conI in the ODS documentation. The ASAM ODSBox uses con_i as API object representing the session. Close this session to release the connection license. Otherwise the session will be auto closed after 30 minutes of inactivity.

In [6]:
try:  
  import odsbox
except:
  !pip install -U odsbox

from odsbox.con_i import ConI
from odsbox.submatrix_to_pandas import submatrix_to_pandas

con_i = ConI(url='https://docker.peak-solution.de:10032/api', auth=('Demo','mdm'))

Exception ignored in: <function ConI.__del__ at 0x0000027D833A2200>
Traceback (most recent call last):
  File "c:\Peak\github\data_management_learning_path\.venv\Lib\site-packages\odsbox\con_i.py", line 136, in __del__
    self.logout()
  File "c:\Peak\github\data_management_learning_path\.venv\Lib\site-packages\odsbox\con_i.py", line 168, in logout
    self.__check_result(response)
  File "c:\Peak\github\data_management_learning_path\.venv\Lib\site-packages\odsbox\con_i.py", line 510, in __check_result
    raise requests.HTTPError(
requests.exceptions.HTTPError: {
  "minorCode": 3,
  "reason": "Cannot find session for connection identifier 9315ec67-7879-4e8c-b768-2f6feeabd582!"
}


## Simple query for some data

First we query for a submatrix containing the bulk data structure of a certain measurement. 
We use this submatrix to query for bulk data in the next step - directly converted to a Dataframe ...

Try different query options (there must be more than an asterix (*) 😉) ... 

In [7]:
# find submatrix (bulk root) of a certain measurement
submatrices = con_i.query_data(
    {
        "AoSubmatrix": {"measurement.name": {"$like": "*"}},
        "$attributes": {"name": 1, "id": 1},
        "$options": {"$rowlimit": 50},
    }
)

# get the bulk data of first submatrix
df = submatrix_to_pandas(con_i, submatrices.iloc[0,1])

if "Time" in df.columns:
    df.set_index("Time", inplace=True)
elif "time" in df.columns:
    df.set_index("time", inplace=True)

df.head()

,U_q,Coolant,Stator_winding,U_d,Stator_tooth,Motor_speed,I_d,I_q,Pm,Stator_yoke,Ambient,Torque
Time,,,,,,,,,,,,
0.0,-1.870446,78.804928,53.393323,0.627027,55.588612,2.761764,-2.001388,1.094317,49.695740,60.426506,25.321585,3.138413e-13
0.5,3.747699,78.859141,53.460695,-2.550798,55.659321,102.164216,-6.588812,21.266870,49.689940,60.490822,25.308411,1.465798e+01
1.0,13.771337,78.900886,53.480459,-9.704846,55.698835,314.802839,-11.969288,41.992287,49.675229,60.542482,25.295898,2.993536e+01
1.5,26.851160,78.939788,53.451930,-19.583736,55.723997,608.984106,-16.067978,57.526612,49.686365,60.578285,25.275402,4.141312e+01
2.0,42.088467,78.966302,53.467553,-31.885562,55.743452,961.552082,-19.428698,69.952405,49.692206,60.605028,25.263575,5.064337e+01


## License

Copyright © 2025 [Peak Solution GmbH](https://peak-solution.de)

The training material in this repository is licensed under a Creative Commons BY-NC-SA 4.0 license. See [LICENSE](../LICENSE) file for more information.